# Parameter Sloppiness Analysis: Chen 2004 Yeast Cell Cycle Model

This notebook computes parameter sloppiness in the Chen et al 2004 budding yeast cell cycle model by analyzing the Hessian eigenvalue spectrum. Parameter sloppiness quantifies how sensitive model outputs are to parameter changes, revealing which parameter combinations matter most for model behavior.

## Approach:
1. **Sensitivity Analysis**: Compute first-order parameter sensitivities S_i(t) = ∂x/∂θ_i using variational equations
2. **Jacobian Construction**: Build Jacobian matrix J where J[k,i] = g_i(t_k) for readout function g_i(t) = ∂y/∂θ_i  
3. **Hessian Eigenvalues**: Form Gauss-Newton Hessian H = J.T @ J and compute eigenvalues
4. **Sloppiness Visualization**: Plot log10(λ_i) vs index to reveal exponentially spaced eigenvalue spectrum

**Expected Result**: Sloppy models show eigenvalues spanning many orders of magnitude, indicating that some parameter combinations are much more important than others.

In [ ]:
# === SECTION 1: LOAD MODEL AND SETUP ===
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import tellurium as te
import roadrunner
import os
import time
import datetime
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
roadrunner.Logger.setLevel(roadrunner.Logger.LOG_CRITICAL)

print("=== Parameter Sloppiness Analysis: Chen 2004 Yeast Cell Cycle ===")
print("Setting up model and integration parameters...")

# Cross-platform path detection for Chen model
def get_model_path():
    """Get the correct path to the SBML model file"""
    possible_paths = [
        "/home/gijs/Documents/OxfordEvolution/Yeast/Chen/chen2004_biomd56.xml",
        "/Users/gijsbartholomeus/Documents/STUDIE/OxfordEvolution/code/Yeast/Chen/chen2004_biomd56.xml",
        "chen2004_biomd56.xml",
        "Chen/chen2004_biomd56.xml",
        "../Chen/chen2004_biomd56.xml"
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            return path
    
    raise FileNotFoundError(f"Chen model not found in expected locations: {possible_paths}")

# Load Chen 2004 budding yeast cell cycle model
model_path = get_model_path()
print(f"Loading Chen model from: {model_path}")
rr = te.loadSBMLModel(model_path)

# Integration parameters for high precision sensitivity analysis
INTEGRATION_PARAMS = {
    'rtol': 1e-7,      # Relative tolerance (tight for sensitivity analysis)
    'atol': 1e-9,      # Absolute tolerance (tight for sensitivity analysis) 
    'method': 'DOP853' # High-order Runge-Kutta method for smooth solutions
}

# Time span for analysis
T_FINAL = 200.0    # Final integration time (minutes)
N_TIME_POINTS = 101  # Number of time points for Jacobian evaluation

# Get model dimensions (will be updated after parameter filtering)
n_species = len(rr.getFloatingSpeciesIds())
n_params_total = len(rr.getGlobalParameterIds())

print(f"Model loaded successfully:")
print(f"  - Species: {n_species}")
print(f"  - Total parameters: {n_params_total} (will filter for settable ones)")
print(f"  - Integration time: [0, {T_FINAL}] minutes")
print(f"  - Time points for Jacobian: {N_TIME_POINTS}")
print(f"  - Integration tolerances: rtol={INTEGRATION_PARAMS['rtol']}, atol={INTEGRATION_PARAMS['atol']}")

# Get kinetic parameters only (same filtering as CPUheavy notebook)
def get_kinetic_parameters(rr):
    """Get list of kinetic parameters, excluding regulatory switches/flags (matches CPUheavy filtering)."""
    kinetic_params = []
    excluded_params = []
    
    for pid in rr.getGlobalParameterIds():
        value = rr.getValue(pid)
        param_lower = pid.lower()
        
        # Exclude non-kinetic parameters (switches, flags, totals) - same logic as CPUheavy
        if (param_lower.endswith('t') and value in [0.0, 1.0]) or \
           (param_lower.startswith('d') and param_lower.endswith('n')) or \
           ('flag' in param_lower) or \
           ('switch' in param_lower) or \
           (value == 0.0) or \
           (pid in ['cell']) or \
           ('total' in param_lower and value in [0.0, 1.0]):
            excluded_params.append(pid)
        else:
            kinetic_params.append(pid)
    
    return kinetic_params, excluded_params

def get_settable_kinetic_parameters(rr):
    """Get kinetic parameters that can also be set independently (exclude assignment rules)."""
    # First get kinetic parameters
    kinetic_params, excluded_kinetic = get_kinetic_parameters(rr)
    
    # Then filter for settability
    settable_kinetic = []
    excluded_assignment = []
    
    for pid in kinetic_params:
        try:
            # Try to set the parameter to test if it's settable
            original_value = rr.getValue(pid)
            rr.setValue(pid, original_value)  # This should work if settable
            settable_kinetic.append(pid)
        except RuntimeError:
            # Parameter is defined by assignment rule or otherwise not settable
            excluded_assignment.append(pid)
    
    return settable_kinetic, excluded_kinetic, excluded_assignment

kinetic_param_names, excluded_nonkinetic, excluded_assignment = get_settable_kinetic_parameters(rr)
print(f"\nParameter filtering results (matching CPUheavy approach):")
print(f"  - Total parameters: {len(rr.getGlobalParameterIds())}")
print(f"  - Kinetic parameters: {len(rr.getGlobalParameterIds()) - len(excluded_nonkinetic)}")
print(f"  - Settable kinetic parameters: {len(kinetic_param_names)}")
print(f"  - Excluded non-kinetic: {len(excluded_nonkinetic)} (switches, flags, totals)")
print(f"  - Excluded assignment rules: {len(excluded_assignment)}")

if excluded_nonkinetic:
    print(f"  - Non-kinetic excluded: {excluded_nonkinetic[:10]}{'...' if len(excluded_nonkinetic) > 10 else ''}")
if excluded_assignment:
    print(f"  - Assignment rule excluded: {excluded_assignment}")

# Store nominal parameter vector θ0 for settable kinetic parameters only
theta0 = np.array([rr.getValue(pid) for pid in kinetic_param_names])
param_names = kinetic_param_names

print(f"\nNominal parameter vector θ0 (settable kinetic parameters only):")
for i, (name, value) in enumerate(zip(param_names, theta0)):
    if i < 10:  # Show first 10 to avoid clutter
        print(f"  θ{i:2d}: {name:12s} = {value:8.4f}")
    elif i == 10:
        print(f"  ... and {len(param_names) - 10} more parameters")

print(f"\nSetup complete! Ready for sensitivity analysis with {len(param_names)} settable kinetic parameters.")
print(f"This should match CPUheavy's 156 kinetic parameters (minus any assignment rule conflicts).")

=== Parameter Sloppiness Analysis: Chen 2004 Yeast Cell Cycle ===
Setting up model and integration parameters...
Loading Chen model from: /home/gijs/Documents/OxfordEvolution/Yeast/Chen/chen2004_biomd56.xml
Model loaded successfully:
  - Species: 50
  - Total parameters: 163 (will filter for settable ones)
  - Integration time: [0, 200.0] minutes
  - Time points for Jacobian: 101
  - Integration tolerances: rtol=1e-07, atol=1e-09

Parameter filtering results (matching CPUheavy approach):
  - Total parameters: 163
  - Kinetic parameters: 156
  - Settable kinetic parameters: 136
  - Excluded non-kinetic: 7 (switches, flags, totals)
  - Excluded assignment rules: 20
  - Non-kinetic excluded: ['CDC15T', 'ESP1T', 'IET', 'kdirent', 'ksn2_p', 'kspds_p', 'TEM1T']
  - Assignment rule excluded: ['D', 'mu', 'Vdb5', 'Vdb2', 'Vasbf', 'Visbf', 'Vkpc1', 'Vkpf6', 'Vacdh', 'Vicdh', 'Vppnet', 'Vkpnet', 'Vdppx', 'Vdpds', 'Vaiep', 'Vd2c1', 'Vd2f6', 'Vppc1', 'Vppf6', 'F']

Nominal parameter vector θ0 (sett

In [3]:
# === SECTION 2: DEFINE ODE SYSTEM WITH PARAMETER DERIVATIVES ===

def get_settable_parameters(rr):
    """Get list of parameters that can be set independently (exclude assignment rules)."""
    settable_params = []
    excluded_params = []
    
    for pid in rr.getGlobalParameterIds():
        try:
            # Try to set the parameter to test if it's settable
            original_value = rr.getValue(pid)
            rr.setValue(pid, original_value)  # This should work if settable
            settable_params.append(pid)
        except RuntimeError:
            # Parameter is defined by assignment rule or otherwise not settable
            excluded_params.append(pid)
    
    return settable_params, excluded_params

def get_settable_species(rr):
    """Get list of species that can be set independently (exclude assignment rules)."""
    settable_species = []
    excluded_species = []
    
    for species_id in rr.getFloatingSpeciesIds():
        try:
            # Try to set the species to test if it's settable
            original_value = rr.getValue(species_id)
            rr.setValue(species_id, original_value)  # This should work if settable
            settable_species.append(species_id)
        except RuntimeError:
            # Species is defined by assignment rule or otherwise not settable
            excluded_species.append(species_id)
    
    return settable_species, excluded_species

class YeastODESystem:
    """
    Wrapper for Chen 2004 yeast cell cycle model with parameter derivatives.
    
    This class provides the ODE system f(x, θ), its Jacobians f_x and f_θ,
    the readout function h(x), and its derivative h_x(x) needed for 
    sensitivity analysis.
    
    Only uses settable parameters and species (excludes those defined by assignment rules).
    """
    
    def __init__(self, roadrunner_model, param_names=None):
        self.rr = roadrunner_model
        
        # Filter to only settable parameters if not provided
        if param_names is None:
            self.param_names, excluded_params = get_settable_parameters(self.rr)
            print(f"Filtered parameters: {len(self.param_names)} settable, {len(excluded_params)} excluded")
            if excluded_params:
                print(f"  Excluded parameters (assignment rules): {excluded_params[:5]}{'...' if len(excluded_params) > 5 else ''}")
        else:
            # Use provided parameter names (assume they're settable)
            self.param_names = param_names
            
        # Filter to only settable species (important for Chen model!)
        self.species_names, excluded_species = get_settable_species(self.rr)
        print(f"Filtered species: {len(self.species_names)} settable, {len(excluded_species)} excluded")
        if excluded_species:
            print(f"  Excluded species (assignment rules): {excluded_species}")
            
        self.n_species = len(self.species_names)
        self.n_params = len(self.param_names)
        
        # Find CLB2 index for readout function
        try:
            self.clb2_index = self.species_names.index('CLB2')
            print(f"Using CLB2 (species index {self.clb2_index}) as readout y(t)")
        except ValueError:
            print("CLB2 not found, using first settable species as readout")
            self.clb2_index = 0
    
    def f(self, t, x, theta):
        """
        ODE system: dx/dt = f(x, θ)
        
        Args:
            t: time
            x: state vector (settable species concentrations only)
            theta: parameter vector
            
        Returns:
            dxdt: time derivatives for settable species only
        """
        # Set current parameters
        for i, param_id in enumerate(self.param_names):
            self.rr.setValue(param_id, theta[i])
        
        # Set current state for settable species only
        for i, species_id in enumerate(self.species_names):
            self.rr.setValue(species_id, x[i])
        
        # Get derivatives from roadrunner
        try:
            # Get all species rates
            all_rates = self.rr.getStateVectorRate()
            all_species = self.rr.getFloatingSpeciesIds()
            
            # Extract rates only for settable species
            rates = []
            for species_id in self.species_names:
                species_idx = all_species.index(species_id)
                rates.append(all_rates[species_idx])
            
            return np.array(rates)
        except:
            # Fallback: get rates manually for settable species
            rates = []
            for species_id in self.species_names:
                try:
                    # Try to get reaction rate for this species
                    rate = self.rr.getValue(f"_RATE_RULE_{species_id}") if f"_RATE_RULE_{species_id}" in self.rr.keys() else 0
                    rates.append(rate)
                except:
                    rates.append(0.0)
            return np.array(rates)
    
    def f_x(self, t, x, theta):
        """
        Jacobian with respect to state: ∂f/∂x
        
        Returns:
            J_x: n_species × n_species Jacobian matrix
        """
        # Use finite differences for Jacobian computation
        eps = 1e-8
        f0 = self.f(t, x, theta)
        J = np.zeros((self.n_species, self.n_species))
        
        for i in range(self.n_species):
            x_pert = x.copy()
            x_pert[i] += eps
            f_pert = self.f(t, x_pert, theta)
            J[:, i] = (f_pert - f0) / eps
            
        return J
    
    def f_theta(self, t, x, theta):
        """
        Jacobian with respect to parameters: ∂f/∂θ
        
        Returns:
            J_theta: n_species × n_params Jacobian matrix
        """
        # Use finite differences for parameter derivatives
        eps = 1e-8
        f0 = self.f(t, x, theta)
        J = np.zeros((self.n_species, self.n_params))
        
        for i in range(self.n_params):
            theta_pert = theta.copy()
            # Use relative perturbation for better numerical stability
            eps_rel = eps * max(abs(theta[i]), 1e-6)
            theta_pert[i] += eps_rel
            f_pert = self.f(t, x, theta_pert)  # Fixed: use x, not x_pert
            J[:, i] = (f_pert - f0) / eps_rel
            
        return J
    
    def h(self, x):
        """
        Readout function: y = h(x)
        Here we use CLB2 concentration as the scalar readout.
        
        Args:
            x: state vector
            
        Returns:
            y: scalar readout (CLB2 concentration)
        """
        return x[self.clb2_index]
    
    def h_x(self, x):
        """
        Derivative of readout function: ∂h/∂x
        
        Args:
            x: state vector
            
        Returns:
            dh_dx: gradient vector (1 × n_species)
        """
        grad = np.zeros(self.n_species)
        grad[self.clb2_index] = 1.0  # ∂(CLB2)/∂x_CLB2 = 1
        return grad
    
    def get_initial_conditions(self, theta):
        """
        Get initial conditions for given parameters.
        Uses model's default steady state or specified initial conditions.
        
        Args:
            theta: parameter vector
            
        Returns:
            x0: initial state vector (settable species only)
        """
        # Set parameters
        for i, param_id in enumerate(self.param_names):
            self.rr.setValue(param_id, theta[i])
        
        # Reset to initial conditions
        self.rr.reset()
        
        # Get initial concentrations for settable species only
        x0 = np.array([self.rr.getValue(species_id) for species_id in self.species_names])
        return x0

# Create ODE system wrapper with settable kinetic parameters only
ode_system = YeastODESystem(rr, param_names)

print("ODE system wrapper created:")
print(f"  - State dimension: {ode_system.n_species}")  
print(f"  - Parameter dimension: {ode_system.n_params} (settable kinetic parameters)")
print(f"  - Readout function: h(x) = {ode_system.species_names[ode_system.clb2_index]}")
print(f"  - Filtered out: {len(excluded_nonkinetic)} non-kinetic + {len(excluded_assignment)} assignment rule parameters")
print(f"  - Note: CPUheavy uses {156} kinetic parameters, we use {len(param_names)} settable kinetic parameters")

# Test the system with nominal parameters
x0 = ode_system.get_initial_conditions(theta0)
print(f"\nInitial conditions x0:")
for i, (name, value) in enumerate(zip(ode_system.species_names, x0)):
    print(f"  x{i:2d}: {name:12s} = {value:8.4f}")

# Test function evaluations
print(f"\nTesting function evaluations at t=0:")
f0 = ode_system.f(0, x0, theta0)
print(f"  f(0, x0, θ0): max(|dxdt|) = {np.max(np.abs(f0)):.2e}")

h0 = ode_system.h(x0)
print(f"  h(x0) = {h0:.4f}")

print(f"\nODE system ready for sensitivity analysis!")

Filtered species: 39 settable, 11 excluded
  Excluded species (assignment rules): ['BCK2', 'CDC14T', 'CDC6T', 'CKIT', 'CLB2T', 'CLB5T', 'CLN3', 'MCM1', 'NET1T', 'SBF', 'SIC1T']
Using CLB2 (species index 14) as readout y(t)
ODE system wrapper created:
  - State dimension: 39
  - Parameter dimension: 136 (settable kinetic parameters)
  - Readout function: h(x) = CLB2
  - Filtered out: 7 non-kinetic + 20 assignment rule parameters
  - Note: CPUheavy uses 156 kinetic parameters, we use 136 settable kinetic parameters

Initial conditions x0:
  x 0: BUB2         =   0.2000
  x 1: BUD          =   0.0085
  x 2: C2           =   0.2384
  x 3: C2P          =   0.0240
  x 4: C5           =   0.0701
  x 5: C5P          =   0.0069
  x 6: CDC14        =   0.4683
  x 7: CDC15        =   0.6565
  x 8: CDC20        =   0.4443
  x 9: CDC20i       =   1.4720
  x10: CDC6         =   0.1076
  x11: CDC6P        =   0.0155
  x12: CDH1         =   0.9305
  x13: CDH1i        =   0.0695
  x14: CLB2         =  

In [ ]:
# === SECTION 3: INTEGRATE SYSTEM WITH SENSITIVITY EQUATIONS ===

def integrate_with_sensitivities(ode_system, theta, t_span, n_points=101, batch_size=None):
    """
    Integrate ODE system with parameter sensitivities.
    
    Solves the combined system:
    - dx/dt = f(x, θ)                    [original ODEs]
    - dS/dt = f_x(x, θ) S + f_θ(x, θ)   [variational equations]
    
    Where S_i(t) = ∂x/∂θ_i are the parameter sensitivities.
    
    Args:
        ode_system: YeastODESystem instance
        theta: parameter vector
        t_span: (t_start, t_end) time interval
        n_points: number of time points for output
        batch_size: if provided, integrate sensitivities in batches to save memory
        
    Returns:
        t: time points
        x: state trajectories (n_points × n_species)
        S: sensitivity matrices (n_points × n_species × n_params)
    """
    start_time = time.time()
    
    print(f"Integrating system with sensitivities...")
    print(f"  Time span: [{t_span[0]:.1f}, {t_span[1]:.1f}]")
    print(f"  Output points: {n_points}")
    
    n_species = ode_system.n_species
    n_params = ode_system.n_params
    
    if batch_size is None:
        batch_size = n_params  # Process all parameters at once
    
    # Time points for evaluation
    t_eval = np.linspace(t_span[0], t_span[1], n_points)
    
    # Initialize results
    x_result = np.zeros((n_points, n_species))
    S_result = np.zeros((n_points, n_species, n_params))
    
    # First, solve the original ODE system
    print(f"  Step 1: Solving original ODEs...")
    step1_start = time.time()
    
    def ode_rhs(t, y):
        """Right-hand side for original ODEs"""
        return ode_system.f(t, y, theta)
    
    # Get initial conditions
    x0 = ode_system.get_initial_conditions(theta)
    
    # Solve original system
    sol_x = solve_ivp(
        ode_rhs, t_span, x0, 
        t_eval=t_eval,
        **INTEGRATION_PARAMS
    )
    
    if not sol_x.success:
        raise RuntimeError(f"ODE integration failed: {sol_x.message}")
    
    x_result = sol_x.y.T  # Transpose to get (n_points × n_species)
    t = sol_x.t
    
    step1_time = time.time() - step1_start
    print(f"    Original ODEs solved successfully in {step1_time:.1f}s")
    print(f"    Solution range: min={np.min(x_result):.2e}, max={np.max(x_result):.2e}")
    
    # Now solve sensitivity equations in batches
    print(f"  Step 2: Solving sensitivity equations...")
    step2_start = time.time()
    
    n_batches = (n_params + batch_size - 1) // batch_size
    print(f"    Processing {n_params} parameters in {n_batches} batches (size {batch_size})")
    print(f"    Estimated time: {step1_time * n_batches:.1f}s - {step1_time * n_batches * 2:.1f}s")
    print()
    
    for batch_idx in range(n_batches):
        batch_start = time.time()
        start_param = batch_idx * batch_size
        end_param = min(start_param + batch_size, n_params)
        batch_n_params = end_param - start_param
        
        # Calculate progress and ETA with actual clock time
        elapsed_total = time.time() - start_time
        if batch_idx > 0:
            avg_batch_time = (time.time() - step2_start) / batch_idx
            remaining_batches = n_batches - batch_idx
            eta_seconds = avg_batch_time * remaining_batches
            
            # Format ETA nicely
            if eta_seconds > 3600:  # > 1 hour
                eta_hours = int(eta_seconds // 3600)
                eta_mins = int((eta_seconds % 3600) // 60)
                eta_time_str = f"{eta_hours}h{eta_mins}m"
            elif eta_seconds > 60:  # > 1 minute
                eta_mins = int(eta_seconds // 60)
                eta_secs = int(eta_seconds % 60)
                eta_time_str = f"{eta_mins}m{eta_secs}s"
            else:
                eta_time_str = f"{eta_seconds:.0f}s"
            
            # Calculate actual completion time
            import datetime
            eta_clock = datetime.datetime.now() + datetime.timedelta(seconds=eta_seconds)
            eta_str = f" | ETA: {eta_time_str} ({eta_clock.strftime('%H:%M:%S')})"
        else:
            eta_str = ""
        
        progress_pct = (batch_idx / n_batches) * 100
        print(f"    [{progress_pct:5.1f}%] Batch {batch_idx + 1}/{n_batches}: params {start_param}-{end_param-1}{eta_str}")
        
        def sensitivity_rhs(t, y_combined):
            """
            Combined RHS for sensitivity equations in this batch.
            y_combined = [S_i1, S_i2, ..., S_ik] where each S_ij is n_species long
            """
            # Interpolate state at current time
            if x_result.ndim > 1:
                x_interp = np.array([np.interp(t, sol_x.t, x_result[:, i]) for i in range(n_species)])
            else:
                x_interp = np.interp(t, sol_x.t, x_result)
            
            # Reshape sensitivity variables
            S_batch = y_combined.reshape((batch_n_params, n_species)).T  # (n_species × batch_n_params)
            
            # Compute Jacobians
            J_x = ode_system.f_x(t, x_interp, theta)      # (n_species × n_species)
            J_theta = ode_system.f_theta(t, x_interp, theta)  # (n_species × n_params)
            
            # Sensitivity equations: dS/dt = J_x @ S + J_theta[:, batch_params]
            batch_J_theta = J_theta[:, start_param:end_param]  # (n_species × batch_n_params)
            dSdt = J_x @ S_batch + batch_J_theta  # (n_species × batch_n_params)
            
            return dSdt.T.flatten()  # Flatten to match input format
        
        # Initial conditions for sensitivities (∂x/∂θ at t=0)
        S0_batch = np.zeros((batch_n_params, n_species))
        
        # If initial conditions depend on parameters, compute ∂x0/∂θ
        # For now, assume x0 is independent of θ, so S0 = 0
        
        # Solve sensitivity equations for this batch
        sol_S_batch = solve_ivp(
            sensitivity_rhs, t_span, S0_batch.flatten(),
            t_eval=t_eval,
            **INTEGRATION_PARAMS
        )
        
        if not sol_S_batch.success:
            batch_time = time.time() - batch_start
            print(f"      ❌ Batch failed in {batch_time:.1f}s: {sol_S_batch.message}")
            # Fill with zeros as fallback
            S_result[:, :, start_param:end_param] = 0
        else:
            batch_time = time.time() - batch_start
            # Reshape and store results
            S_batch_result = sol_S_batch.y.reshape((batch_n_params, n_species, n_points))
            S_result[:, :, start_param:end_param] = S_batch_result.transpose(2, 1, 0)
            print(f"      ✅ Batch completed in {batch_time:.1f}s")
    
    total_time = time.time() - start_time
    step2_time = time.time() - step2_start
    
    print(f"\n  🎉 Integration complete!")
    print(f"    Total time: {total_time:.1f}s ({total_time/60:.1f}m)")
    print(f"    Step 1 (ODEs): {step1_time:.1f}s")  
    print(f"    Step 2 (Sensitivities): {step2_time:.1f}s")
    print(f"    Time points: {len(t)}")
    print(f"    State shape: {x_result.shape}")
    print(f"    Sensitivity shape: {S_result.shape}")
    print(f"    Max sensitivity magnitude: {np.max(np.abs(S_result)):.2e}")
    
    return t, x_result, S_result

# Test integration with a shorter time span first
print("🧪 Testing integration with sensitivities...")
print("This will help estimate timing for the full analysis.")
t_test = np.array([0, 50])  # Shorter time span for testing

t, x, S = integrate_with_sensitivities(
    ode_system, theta0, t_test, 
    n_points=21,  # Fewer points for testing
    batch_size=10  # Reasonable batch size for progress tracking
)
print(f"Results summary:")
print(f"  - Time range: [{t[0]:.1f}, {t[-1]:.1f}]")
print(f"  - State (x) range: [{np.min(x):.2e}, {np.max(x):.2e}]")
print(f"  - Sensitivity (S) range: [{np.min(S):.2e}, {np.max(S):.2e}]")
print(f"  - Largest sensitivity magnitude: {np.max(np.abs(S)):.2e}")

# Quick sanity check: CLB2 trajectory
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(t, x[:, ode_system.clb2_index], 'b-', linewidth=2)
plt.xlabel('Time (min)')
plt.ylabel('CLB2 Concentration')
plt.title('CLB2 Trajectory')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Plot sensitivities of CLB2 to first few parameters
for i in range(min(5, ode_system.n_params)):
    plt.plot(t, S[:, ode_system.clb2_index, i], label=f'∂CLB2/∂θ{i}')
plt.xlabel('Time (min)')
plt.ylabel('Sensitivity')
plt.title('CLB2 Parameter Sensitivities')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Integration test successful! Ready for full analysis with progress tracking.")

🧪 Testing integration with sensitivities...
This will help estimate timing for the full analysis.
Integrating system with sensitivities...
  Time span: [0.0, 50.0]
  Output points: 21
  Step 1: Solving original ODEs...
    Original ODEs solved successfully in 298.1s
    Solution range: min=7.24e-05, max=1.47e+00
  Step 2: Solving sensitivity equations...
    Processing 136 parameters in 14 batches (size 10)
    Estimated time: 4173.4s - 8346.8s

    [  0.0%] Batch 1/14: params 0-9
    Original ODEs solved successfully in 298.1s
    Solution range: min=7.24e-05, max=1.47e+00
  Step 2: Solving sensitivity equations...
    Processing 136 parameters in 14 batches (size 10)
    Estimated time: 4173.4s - 8346.8s

    [  0.0%] Batch 1/14: params 0-9


In [ ]:
# === SECTION 4: BUILD JACOBIAN MATRIX ===

def build_jacobian(ode_system, t, x, S):
    """
    Build the Jacobian matrix J for parameter sloppiness analysis.
    
    For each time point t_k and parameter θ_i, computes:
    g_i(t_k) = ∂y/∂θ_i = h_x(x(t_k)) · S_i(t_k)
    
    Where:
    - y(t) = h(x(t)) is the scalar readout function
    - S_i(t_k) = ∂x/∂θ_i(t_k) is the sensitivity of state to parameter i
    - h_x is the gradient of the readout function
    
    Args:
        ode_system: YeastODESystem instance
        t: time points (n_points,)
        x: state trajectories (n_points × n_species) 
        S: sensitivity matrices (n_points × n_species × n_params)
        
    Returns:
        J: Jacobian matrix (n_points × n_params)
        y: readout values y(t) = h(x(t)) (n_points,)
    """
    print("Building Jacobian matrix J...")
    
    n_points, n_species, n_params = S.shape
    
    # Initialize outputs
    J = np.zeros((n_points, n_params))
    y = np.zeros(n_points)
    
    # Compute Jacobian entries
    for k in range(n_points):
        # Current state
        x_k = x[k, :]
        
        # Readout value: y(t_k) = h(x(t_k))
        y[k] = ode_system.h(x_k)
        
        # Readout gradient: ∂h/∂x
        h_x_k = ode_system.h_x(x_k)
        
        # Jacobian entries: g_i(t_k) = ∂y/∂θ_i = h_x · S_i(t_k)
        for i in range(n_params):
            S_i_k = S[k, :, i]  # Sensitivity ∂x/∂θ_i at time t_k
            J[k, i] = np.dot(h_x_k, S_i_k)
    
    print(f"  Jacobian shape: {J.shape}")
    print(f"  Readout values (y) range: [{np.min(y):.2e}, {np.max(y):.2e}]")
    print(f"  Jacobian entries range: [{np.min(J):.2e}, {np.max(J):.2e}]")
    print(f"  Largest Jacobian magnitude: {np.max(np.abs(J)):.2e}")
    
    # Check for numerical issues
    n_finite = np.sum(np.isfinite(J))
    n_total = J.size
    if n_finite < n_total:
        print(f"  Warning: {n_total - n_finite}/{n_total} Jacobian entries are non-finite")
    
    return J, y

# Build Jacobian from test integration
print("Building Jacobian matrix from test integration...")
J_test, y_test = build_jacobian(ode_system, t, x, S)

# Visualize Jacobian structure
print("\nJacobian analysis:")
print(f"  Shape: {J_test.shape} (time × parameters)")
print(f"  Condition number estimate: {np.linalg.cond(J_test):.2e}")

# Plot Jacobian heatmap
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(t, y_test, 'b-', linewidth=2)
plt.xlabel('Time (min)')
plt.ylabel('Readout y(t)')
plt.title('Readout Function: CLB2(t)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Show first 20 parameters to avoid overcrowding
n_show = min(20, ode_system.n_params)
im = plt.imshow(J_test[:, :n_show].T, aspect='auto', cmap='RdBu_r', 
               extent=[t[0], t[-1], 0, n_show])
plt.colorbar(im, label='∂y/∂θ_i')
plt.xlabel('Time (min)')
plt.ylabel('Parameter Index')
plt.title(f'Jacobian Matrix J (first {n_show} parameters)')

plt.tight_layout()
plt.show()

# Analyze parameter importance over time
print(f"\nParameter importance analysis (first {min(10, ode_system.n_params)} parameters):")
for i in range(min(10, ode_system.n_params)):
    max_sensitivity = np.max(np.abs(J_test[:, i]))
    rms_sensitivity = np.sqrt(np.mean(J_test[:, i]**2))
    print(f"  θ{i:2d} ({param_names[i]:12s}): max=|{max_sensitivity:.2e}|, RMS={rms_sensitivity:.2e}")

print("\nJacobian construction successful!")

In [ ]:
# === SECTION 5: COMPUTE HESSIAN AND EIGENVALUES ===

def compute_hessian_eigenvalues(J, regularization=1e-12):
    """
    Compute Gauss-Newton Hessian eigenvalues for sloppiness analysis.
    
    Forms H = J.T @ J and computes its eigenvalues, which quantify
    parameter sloppiness. Sloppy models show eigenvalues spanning
    many orders of magnitude.
    
    Args:
        J: Jacobian matrix (n_points × n_params)
        regularization: small value added to diagonal for numerical stability
        
    Returns:
        eigenvalues: sorted eigenvalues (largest to smallest)
        eigenvectors: corresponding eigenvectors
        H: Hessian matrix
        condition_number: condition number of Hessian
    """
    print("Computing Gauss-Newton Hessian eigenvalues...")
    
    n_points, n_params = J.shape
    print(f"  Jacobian shape: {n_points} time points × {n_params} parameters")
    
    # Form Gauss-Newton Hessian: H = J.T @ J
    H = J.T @ J
    
    # Add regularization for numerical stability
    H += regularization * np.eye(n_params)
    
    print(f"  Hessian shape: {H.shape}")
    print(f"  Hessian range: [{np.min(H):.2e}, {np.max(H):.2e}]")
    print(f"  Regularization: {regularization:.2e} added to diagonal")
    
    # Compute eigenvalues and eigenvectors
    # Use eigh for symmetric matrices (more stable than eig)
    eigenvals, eigenvecs = np.linalg.eigh(H)
    
    # Sort eigenvalues in descending order (largest to smallest)
    sort_idx = np.argsort(eigenvals)[::-1]
    eigenvals = eigenvals[sort_idx]
    eigenvecs = eigenvecs[:, sort_idx]
    
    # Compute condition number
    condition_number = eigenvals[0] / eigenvals[-1] if eigenvals[-1] > 0 else np.inf
    
    print(f"  Eigenvalue computation successful!")
    print(f"  Number of eigenvalues: {len(eigenvals)}")
    print(f"  Largest eigenvalue: {eigenvals[0]:.2e}")
    print(f"  Smallest eigenvalue: {eigenvals[-1]:.2e}")
    print(f"  Condition number: {condition_number:.2e}")
    
    # Check for numerical issues
    n_positive = np.sum(eigenvals > 0)
    n_negative = np.sum(eigenvals < 0)
    n_zero = np.sum(np.abs(eigenvals) < 1e-14)
    
    print(f"  Eigenvalue signs: {n_positive} positive, {n_zero} near-zero, {n_negative} negative")
    
    if n_negative > 0:
        print(f"  Warning: {n_negative} negative eigenvalues detected (numerical issues?)")
    
    return eigenvals, eigenvecs, H, condition_number

def analyze_sloppiness(eigenvalues, param_names=None):
    """
    Analyze parameter sloppiness from eigenvalue spectrum.
    
    Args:
        eigenvalues: sorted eigenvalues (largest to smallest)
        param_names: optional parameter names for analysis
        
    Returns:
        sloppiness_metrics: dictionary of sloppiness measures
    """
    print("Analyzing parameter sloppiness...")
    
    # Filter out near-zero eigenvalues
    positive_evals = eigenvalues[eigenvalues > 1e-14]
    n_positive = len(positive_evals)
    
    if n_positive < 2:
        print("  Warning: Insufficient positive eigenvalues for sloppiness analysis")
        return {}
    
    # Sloppiness metrics
    lambda_max = positive_evals[0]
    lambda_min = positive_evals[-1] 
    dynamic_range = lambda_max / lambda_min
    
    # Eigenvalue spacing (geometric mean of ratios)
    ratios = positive_evals[:-1] / positive_evals[1:]
    geometric_mean_ratio = np.exp(np.mean(np.log(ratios)))
    
    # Effective dimensionality (participation ratio)
    sum_evals = np.sum(positive_evals)
    sum_evals_sq = np.sum(positive_evals**2)
    effective_dim = sum_evals**2 / sum_evals_sq if sum_evals_sq > 0 else 0
    
    metrics = {
        'n_parameters': len(eigenvalues),
        'n_positive_eigenvalues': n_positive,
        'lambda_max': lambda_max,
        'lambda_min': lambda_min,
        'dynamic_range': dynamic_range,
        'log10_dynamic_range': np.log10(dynamic_range),
        'geometric_mean_ratio': geometric_mean_ratio,
        'effective_dimensionality': effective_dim,
        'sloppy_fraction': effective_dim / n_positive,
    }
    
    print(f"  Sloppiness Analysis Results:")
    print(f"    Total parameters: {metrics['n_parameters']}")
    print(f"    Positive eigenvalues: {metrics['n_positive_eigenvalues']}")
    print(f"    Dynamic range: {metrics['dynamic_range']:.2e} ({metrics['log10_dynamic_range']:.1f} decades)")
    print(f"    Geometric mean ratio: {metrics['geometric_mean_ratio']:.2f}")
    print(f"    Effective dimensionality: {metrics['effective_dimensionality']:.1f}")
    print(f"    Sloppy fraction: {metrics['sloppy_fraction']:.2f}")
    
    # Interpret sloppiness
    if metrics['log10_dynamic_range'] > 10:
        print(f"    Interpretation: HIGHLY SLOPPY (>10 decades)")
    elif metrics['log10_dynamic_range'] > 5:
        print(f"    Interpretation: MODERATELY SLOPPY (5-10 decades)")
    elif metrics['log10_dynamic_range'] > 2:
        print(f"    Interpretation: MILDLY SLOPPY (2-5 decades)")
    else:
        print(f"    Interpretation: NOT SLOPPY (<2 decades)")
    
    return metrics

# Test Hessian computation on test data
print("Computing Hessian eigenvalues from test Jacobian...")
eigenvals_test, eigenvecs_test, H_test, cond_num_test = compute_hessian_eigenvalues(J_test)

# Analyze sloppiness
metrics_test = analyze_sloppiness(eigenvals_test, param_names)

print(f"\nTest analysis complete!")
print(f"Ready for full integration and sloppiness analysis.")

In [ ]:
# === SECTION 6: PLOT EIGENVALUE SPECTRUM ===

def compute_hessian_and_plot(ode_system, theta, t_span=(0, 200), n_points=101, 
                           batch_size=10, save_plots=True):
    """
    Complete parameter sloppiness analysis: integrate, compute Hessian, and plot eigenvalue spectrum.
    
    This is the main function that ties together all the components for a full sloppiness analysis.
    
    Args:
        ode_system: YeastODESystem instance
        theta: parameter vector
        t_span: time interval for integration
        n_points: number of time points for Jacobian
        batch_size: batch size for sensitivity integration
        save_plots: whether to save plots to file
        
    Returns:
        results: dictionary containing all analysis results
    """
    print("="*70)
    print("COMPLETE PARAMETER SLOPPINESS ANALYSIS")
    print("="*70)
    
    start_time = time.time()
    
    # Step 1: Integrate with sensitivities
    print("\n1. Integrating ODE system with parameter sensitivities...")
    t, x, S = integrate_with_sensitivities(ode_system, theta, t_span, n_points, batch_size)
    integration_time = time.time() - start_time
    print(f"   Integration completed in {integration_time:.1f} seconds")
    
    # Step 2: Build Jacobian
    print("\n2. Building Jacobian matrix...")
    J, y = build_jacobian(ode_system, t, x, S)
    
    # Step 3: Compute Hessian eigenvalues
    print("\n3. Computing Hessian eigenvalues...")
    eigenvals, eigenvecs, H, cond_num = compute_hessian_eigenvalues(J)
    
    # Step 4: Analyze sloppiness
    print("\n4. Analyzing parameter sloppiness...")
    metrics = analyze_sloppiness(eigenvals, ode_system.param_names)
    
    total_time = time.time() - start_time
    print(f"\n   Total analysis time: {total_time:.1f} seconds")
    
    # Step 5: Create comprehensive plots
    print("\n5. Creating sloppiness visualization...")
    
    fig = plt.figure(figsize=(16, 12))
    
    # Plot 1: Eigenvalue spectrum (main sloppiness plot)
    ax1 = plt.subplot(2, 3, 1)
    positive_evals = eigenvals[eigenvals > 1e-14]
    eigenval_indices = np.arange(1, len(positive_evals) + 1)
    
    plt.semilogy(eigenval_indices, positive_evals, 'bo-', markersize=6, linewidth=2)
    plt.xlabel('Eigenvalue Index')
    plt.ylabel('Eigenvalue λᵢ')
    plt.title('Parameter Sloppiness Spectrum\\n(Hessian Eigenvalues)')
    plt.grid(True, alpha=0.3)
    
    # Add dynamic range annotation
    if len(positive_evals) > 1:
        dynamic_range = positive_evals[0] / positive_evals[-1]
        plt.text(0.6, 0.8, f'Dynamic Range:\\n{dynamic_range:.1e}\\n({np.log10(dynamic_range):.1f} decades)', 
                transform=ax1.transAxes, bbox=dict(boxstyle="round,pad=0.3", facecolor="wheat"))
    
    # Plot 2: CLB2 trajectory
    plt.subplot(2, 3, 2)
    plt.plot(t, y, 'r-', linewidth=2)
    plt.xlabel('Time (min)')
    plt.ylabel('CLB2 Concentration')
    plt.title('Readout Function y(t) = CLB2(t)')
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Hessian matrix heatmap
    ax3 = plt.subplot(2, 3, 3)
    # Show submatrix for visualization if too large
    n_show = min(50, H.shape[0])
    H_show = H[:n_show, :n_show]
    im = plt.imshow(np.log10(np.abs(H_show) + 1e-16), cmap='viridis', aspect='auto')
    plt.colorbar(im, label='log₁₀|H|')
    plt.xlabel('Parameter Index')
    plt.ylabel('Parameter Index')
    plt.title(f'Hessian Matrix H = JᵀJ\\n(first {n_show}×{n_show})')
    
    # Plot 4: Jacobian matrix heatmap  
    plt.subplot(2, 3, 4)
    n_show_params = min(30, J.shape[1])
    J_show = J[:, :n_show_params]
    im = plt.imshow(J_show.T, aspect='auto', cmap='RdBu_r', 
                   extent=[t[0], t[-1], 0, n_show_params])
    plt.colorbar(im, label='∂y/∂θᵢ')
    plt.xlabel('Time (min)')
    plt.ylabel('Parameter Index')
    plt.title(f'Jacobian Matrix J\\n(first {n_show_params} parameters)')
    
    # Plot 5: Eigenvalue ratios (sloppiness diagnostic)
    plt.subplot(2, 3, 5)
    if len(positive_evals) > 1:
        ratios = positive_evals[:-1] / positive_evals[1:]
        plt.semilogy(range(1, len(ratios) + 1), ratios, 'go-', markersize=4)
        plt.xlabel('Index i')
        plt.ylabel('Ratio λᵢ/λᵢ₊₁')
        plt.title('Eigenvalue Ratios\\n(Consistency Check)')
        plt.grid(True, alpha=0.3)
        
        # Add geometric mean annotation
        geom_mean = np.exp(np.mean(np.log(ratios)))
        plt.axhline(geom_mean, color='red', linestyle='--', 
                   label=f'Geometric Mean: {geom_mean:.2f}')
        plt.legend()
    
    # Plot 6: Parameter importance (top eigenvector components)
    plt.subplot(2, 3, 6)
    # Show most important eigenvector (largest eigenvalue)
    most_important_evec = np.abs(eigenvecs[:, 0])
    n_show_params = min(20, len(most_important_evec))
    
    param_indices = np.arange(n_show_params)
    plt.bar(param_indices, most_important_evec[:n_show_params])
    plt.xlabel('Parameter Index')
    plt.ylabel('|Eigenvector Component|')
    plt.title('Most Important Parameter Directions\\n(Largest Eigenvalue)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig('chen_parameter_sloppiness_analysis.png', dpi=300, bbox_inches='tight')
        print(f"   Plot saved as: chen_parameter_sloppiness_analysis.png")
    
    plt.show()
    
    # Step 6: Summary report
    print("\n" + "="*70)
    print("SLOPPINESS ANALYSIS SUMMARY")
    print("="*70)
    print(f"Model: Chen et al. 2004 Yeast Cell Cycle")
    print(f"Parameters analyzed: {len(eigenvals)}")
    print(f"Time span: [{t_span[0]:.1f}, {t_span[1]:.1f}] minutes")
    print(f"Time points: {n_points}")
    print(f"Readout function: CLB2 concentration")
    print()
    print(f"Key Results:")
    print(f"  • Dynamic range: {metrics.get('dynamic_range', 0):.2e} ({metrics.get('log10_dynamic_range', 0):.1f} decades)")
    print(f"  • Condition number: {cond_num:.2e}")
    print(f"  • Effective dimensionality: {metrics.get('effective_dimensionality', 0):.1f}")
    print(f"  • Sloppy fraction: {metrics.get('sloppy_fraction', 0):.2f}")
    print()
    
    if metrics.get('log10_dynamic_range', 0) > 5:
        print(f"🎯 CONCLUSION: This model shows SIGNIFICANT PARAMETER SLOPPINESS!")
        print(f"   The eigenvalue spectrum spans {metrics.get('log10_dynamic_range', 0):.1f} decades,")
        print(f"   indicating that parameter combinations exist with vastly different")
        print(f"   importance for the CLB2 readout function.")
    else:
        print(f"📊 CONCLUSION: This model shows LIMITED parameter sloppiness.")
        print(f"   Most parameters have similar importance for the readout function.")
    
    # Package results
    results = {
        'time': t,
        'state_trajectory': x,
        'readout_trajectory': y,
        'sensitivities': S,
        'jacobian': J,
        'hessian': H,
        'eigenvalues': eigenvals,
        'eigenvectors': eigenvecs,
        'condition_number': cond_num,
        'sloppiness_metrics': metrics,
        'integration_time': integration_time,
        'total_time': total_time
    }
    
    return results

# Run full analysis
print("🚀 Running complete parameter sloppiness analysis...")
print("📝 Progress will be tracked with ETA estimates for the sensitivity integration.")
print("⏱️  Note: This may take several minutes for the full model...")
print()

# Full analysis with reasonable parameters
results = compute_hessian_and_plot(
    ode_system, theta0, 
    t_span=(0, T_FINAL),   # Full time span
    n_points=N_TIME_POINTS, # Full resolution
    batch_size=15,         # Reasonable batch size for progress tracking
    save_plots=True
)

print("\n🎉 Parameter sloppiness analysis complete!")
print(f"All results stored in 'results' dictionary for further analysis.")